# Story Teller Workflow Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a story out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and story creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [1]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai==0.95.0
!pip install crewai-tools==0.25.8
!pip install duckduckgo-search
# !pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.6/32.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.3/353.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Step 2: Define the Agents

In [12]:
from crewai import Agent

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    # llm = 'gemini/gemini-2.0-flash-lite',
    llm = 'groq/gemma2-9b-it',
    # llm = 'gpt-4o-mini',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information.\
                  Consider you are on : '+Today),
    tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

## Agent to create a story out of information.
## Back story sets the context for the agent.
story_llm = LLM (model='groq/llama3-70b-8192', temperature=0.8)
# story_llm = LLM (model='gpt-4o-mini', temperature=0.8)
Story_Teller = Agent (
    role = 'Story Teller',
    goal = 'With the details given to you create an interesting story out of it',
    llm=story_llm,
    verbose = True,
    backstory = ('You are very good in creating stories with the given chain of information.\
                 Tell as a comic story in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


### Step 3: Define the Tasks

In [13]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to tell story.
tell_a_Story = Task (
                      description = "Considering the given details in time order make an interesting story",
                      expected_output = "An interesting story connecting key details",
                      agent = Story_Teller,
                      context = [get_details],
                      output_file = 'Story.txt'
                    )



### Step 4: Set Up the Crew

In [14]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Story_Teller],
    tasks = [get_details, tell_a_Story],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [15]:
# Set your OpenAI API key or any other LLM API key
from google.colab import userdata

# Set your Groq API key or any other LLM API key
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Run the crew with a specific stock
result = crew.kickoff (inputs={'topic' : "Recent meeting of Trump with Zelensky"})


# Agent: content explorer
## Task: Get latest, trending, interesting information and news about Recent meeting of Trump with Zelensky
2025-03-09 10:58:03.251247


# Agent: content explorer
## Thought: Thought: I need to find information about a recent meeting between Trump and Zelensky.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"recent meeting between Trump and Zelensky\"}"
## Tool Output: 

Search results: Title: Full Video: Trump-Zelenskyy Oval Office meeting explodes into shouting match
Score: 0.14363066852092743
Url: https://www.cnbc.com/2025/02/28/trump-vance-and-zelenskyy-clash-at-white-house-ukraine-meeting.html
ID: https://www.cnbc.com/2025/02/28/trump-vance-and-zelenskyy-clash-at-white-house-ukraine-meeting.html
---
Title: Contentious Trump-Zelensky meeting threatens U.S. support for Ukraine
Score: 0.14280183613300323
Url: https://www.washingtonpost.com/politics/2025/02/28/trump-ukraine-russia-zelensky/
ID: https://www.washingtonpost.com/politics/

In [11]:
# Alternative Search tool from Crew AI tools. Try this and see the difference
from google.colab import userdata
from crewai_tools import EXASearchTool
import os

os.environ['EXA_API_KEY'] = userdata.get('EXA_API_KEY')

Exa_Search_Tool = EXASearchTool ()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see t


---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
